In [25]:
!pip install textblob_fr

In [26]:
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
import pandas as pd
import spacy
import re

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [27]:
nlp = spacy.load("fr_core_news_sm")

def nlp_pipeline(text):
    text = text.lower().replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"(\s\-\s|-$)", "", text)
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"[&+;#:$@%-]", "", text)
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(lemmas)

try:
    df = pd.read_csv('/kaggle/input/pertinent-data/merged_predictions_with_relevance.csv')
    df['text'] = df['text'].apply(nlp_pipeline)
    print(df.head())
    print(df.describe())
except Exception as e:
    print(f"An error occurred: {e}")


In [28]:
df.head()

,label,id,text,predicted_label,confidence,CLS_Relevance Yes/No
0,NaN,DMAMGYZL53,La proposition du PRD sur la règle de 10%\n\nL...,0,0.857464,No (m)
1,NaN,TTRAZK9XGJ,[ VIDEO ] ------------- (ex-Destiny’s Child) m...,0,0.849603,No (m)
2,NaN,T2YIHR1I2U,"Égypte : La FSBB exprime sa "" préoccupation "" ...",0,0.853480,No (m)
3,NaN,7JS87XQ62T,"(Dossier) Cherté du supercarburant, hausse des...",1,0.705428,Yes (m)
4,NaN,A6CVE0463M,Laser du lundi : Quand ---------- se meut entr...,0,0.855187,Yes (m)


In [30]:
def get_sentiment(text):
    blob = tb(text)
    return blob.sentiment[0]

df['sentiment_score'] = df['text'].apply(get_sentiment)

df.head()
df.describe()

,label,predicted_label,confidence,sentiment_score
count,0.0,9436.000000,9436.000000,9436.000000
mean,NaN,0.040271,0.827305,0.091024
std,NaN,0.196605,0.059281,0.080773
min,NaN,0.000000,0.500042,-1.000000
25%,NaN,0.000000,0.830194,0.046154
50%,NaN,0.000000,0.846076,0.092308
75%,NaN,0.000000,0.855273,0.136851
max,NaN,1.000000,0.872387,0.650000


In [31]:
df.to_csv("Analyse de sentiment 1")